### <font color=green> Import Data

Читаю данные 

In [37]:
import os

from tqdm import tqdm
import datetime

import numpy as np
#from fastparquet import ParquetFile
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
parq = pd.read_parquet('/data/edmitrie/alfa_boosters/parquets/part-00000.parquet')
#parq = ParquetFile(os.path.join(os.getcwd(), 'data', 'train_transaction_contest', 'part-00000.parquet')).to_pandas()

train_target_df = pd.read_csv('alfabattle2_abattle_train_target.csv')
# print(f'Размерность обучающего фрейма: {df.shape}, Размерность сабмит-фрейма {df_submit.shape}, Размерность parquet-файла: {parq.shape}')

In [23]:
parq.head(2)

,timestamp,application_id,client,session_id,event_type,event_category,event_name,event_label,device_screen_name,timezone,device_is_webview,page_urlhost,page_urlpath_full,net_connection_type,net_connection_tech
0,2020-01-15 08:46:32.579,anketa,107efb4507525ce982e19331b36b0e2f,9a1366886da59ec85f0ef5e06f1e97f6,pv,None,None,None,None,Asia/Omsk,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None
1,2020-04-29 19:20:09.705,anketa,32dac9ab28a497a8061e02786bb3a95a,bdf7fc21e4e88d1b4f0838c80b21a1e3,pv,None,None,None,None,Asia/Yekaterinburg,True,anketa.alfabank.ru,03eb3ceab6204f8b602f1863f7fbce01,None,None


### <font color=green> Feature SELECTION

Гипотеза: есть `event_category` указывающие довольно точно на `multiclass-target`, которым размечается сессия.

Нужно отобрать из всего списка категорий операций маленький набор категорий, классы таргета, на которые они указывают, будут обладать минимальной энтропией. Присоединяю к паркету датафрейм с трен-тест-таргетами.

In [24]:
cols_to_check = [col for col in parq.columns if col not in ['client', 'session_id', 'timestamp', 'timezone', 'device_is_webview']]

In [25]:
cols_to_check

['application_id',
 'event_type',
 'event_category',
 'event_name',
 'event_label',
 'device_screen_name',
 'page_urlhost',
 'page_urlpath_full',
 'net_connection_type',
 'net_connection_tech']

In [26]:
%%time
parq = parq.merge(train_target_df[['session_id', 'multi_class_target']], on='session_id', how='left')
parq = parq[parq['multi_class_target'].notnull()]

CPU times: user 11 s, sys: 5.05 s, total: 16.1 s
Wall time: 16.1 s


In [31]:
parq.fillna('nan', inplace=True)

In [32]:
def select_by_count(df, col, threshold=0.005):
    buf = df[['session_id', col]]
    
    sessions_number = buf['session_id'].unique().shape
    
    grouped = buf[col].value_counts()/sessions_number
    
#     print(grouped[grouped > threshold].values * sessions_number)
    
    return grouped[grouped > threshold].index

In [33]:
parq['session_id'].unique().shape[0]

526700

In [42]:
def feature_types_selection(df, col):
    df = df[['session_id', col, 'multi_class_target']]
    
    print("Колонка: ",col)
    print("Было: ", df[col].unique().shape)
        
    df.drop_duplicates(inplace=True)
    
    sessions_number = df['session_id'].unique().shape[0]
    
    df = df[[col, 'multi_class_target']]
    
    if df[col].unique().shape[0] > 700:
        grouped =df[col].value_counts()/sessions_number
        
        cat_to_leave = grouped[grouped > 0.001].index.values.tolist()
        
        df.loc[df[col].apply(lambda x: x not in cat_to_leave), col] = 'other'
        
        print('Стало признаков:', df[col].unique().shape)
    
    cat_df = pd.get_dummies(df, columns=[col],prefix='', prefix_sep='', dummy_na=False)
    cat_feats = [col for col in cat_df.columns if col != 'multi_class_target']
    print(f'Размерность признакового пространства (чило получившихся OneHot-переменны): {len(cat_feats)}')
    
    X, y = cat_df[cat_feats], cat_df['multi_class_target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True, stratify=y)

    rfc = RandomForestClassifier(n_estimators = 300, n_jobs = 64, max_depth = 10, criterion='entropy', min_impurity_decrease = 1e-6, random_state=1)
    rfc.fit(X_train.values, y_train.values)
    
    df_f = pd.DataFrame(zip(cat_feats, rfc.feature_importances_), columns=['f', 'im']).sort_values(by='im', ascending=False)
    
    def get_new_features(df):
        for i in range(df.shape[0]):
            if df['im'].iloc[:i].sum()/df['im'].sum() > 0.99:
                break
        return df['f'].iloc[:i].values.tolist()
    
    atrs = get_new_features(df_f)
    
    print("Осталось:", len(atrs))
    
    return get_new_features(df_f)

In [43]:
dict_atrs_usefull_ctgrs = {col:feature_types_selection(parq, col) for col in  cols_to_check}

Колонка:  application_id
Было:  (22,)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Размерность признакового пространства (чило получившихся OneHot-переменны): 22
Осталось: 13
Колонка:  event_type
Было:  (5,)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Размерность признакового пространства (чило получившихся OneHot-переменны): 5
Осталось: 4
Колонка:  event_category
Было:  (302,)


...

Стало признаков: (6,)
Размерность признакового пространства (чило получившихся OneHot-переменны): 6
Осталось: 5
Колонка:  net_connection_type
Было:  (4,)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Размерность признакового пространства (чило получившихся OneHot-переменны): 4
Осталось: 3
Колонка:  net_connection_tech
Было:  (14,)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Размерность признакового пространства (чило получившихся OneHot-переменны): 14
Осталось: 11


In [44]:
for col in dict_atrs_usefull_ctgrs:
    print(col)
    print(len(dict_atrs_usefull_ctgrs[col]))

application_id
13
event_type
4
event_category
30
event_name
49
event_label
22
device_screen_name
57
page_urlhost
5
page_urlpath_full
5
net_connection_type
3
net_connection_tech
11


In [45]:
import pickle

In [46]:
pickle.dump(dict_atrs_usefull_ctgrs, open('dict_usefull_ctgrs', 'wb'))